# Modelo seq2seq. Construcción de un traductor

## Referencias

1. Basado en F. Collet, [Character-level recurrent sequence-to-sequence model](https://keras.io/examples/nlp/lstm_seq2seq/)
2. [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)
3. Ilya Sutskever et al. (Google),[Sequence to Sequence Learning
with Neural Network](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)

## Contenido

* [Importa módulos](#Importa-módulos)
* [Configuración](#Configuración)
* [Prepara los datos](#Prepara-los-datos)
* [Construye el modelo](#Construye-el-modelo)
* [Entrena el modelo](#Entrena-el-modelo)
* [Modelo de inferencia](#Modelo-de-inferencia)
* [Prueba del modelo](#Prueba-del-modelo)


## Introducción

El modelo de secuencia a secuencia (seq2seq)  es un modelo de aprendizaje que convierte una secuencia de entrada en una secuencia de salida. 

En este contexto, la secuencia es una lista de símbolos, correspondiente a las palabras en una oración. 

El modelo seq2seq ha logrado un gran éxito en campos como la traducción automática, los sistemas de diálogo, la respuesta a preguntas y el resumen de texto. 

Todas estas tareas pueden considerarse como la tarea de aprender un modelo que convierte una secuencia de entrada en una secuencia de salida.

La imagen muestra la arquitectura general del modelo.

<figure>
<center>
<img src="../Imagenes/seq2seq.png" width="500" height="300" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Arquitectura del modelo seq2seq</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

El modelo consta escencialmente de las siguientes capas.

1. Capa de incrustación de codificador
2. Capa recurrente del codificador
3. Capa de incrustación de decodificador
4. Capa recurrente del decodificador
5. Capa de salida del decodificador

Veámos la implementación en t.keras propuesta por F. Collet, [Character-level recurrent sequence-to-sequence model].

## Importa módulos


In [1]:
import numpy as np
import tensorflow as tf
#from tensorflow import keras

from tensorflow.keras.layers import Input, LSTM, Dense

from tensorflow.keras.models import Model

from tensorflow.keras.utils import plot_model

## Descarga los datos

Puede descargar los de [Tab-delimited Bilingual Sentence Pairs](http://www.manythings.org/anki/)

## Configuración


In [5]:
batch_size = 64 # tamño de los lotes para entrenamiento
epochs = 100 # Número de epochs
latent_dim = 256 # dimensión del espacio latente para el encoder
num_samples = 10000
# path del archivo
data_path = "spa-eng/spa.txt" # 124548 lines

## Prepara los datos


In [7]:
# Vectoriza los datos
#
i=0
targe_text= ''

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n") # 124548 lines

for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, target_text, _ = line.split("\t")
    if (i<20):
        print("I: " + input_text + "- T: " + target_text)
    # Usaremos "tab" como el  caracter de inicio (start sequence)
    # para los targets, y "\n" como el caracter de fin de secuencia "end sequence"
    target_text = "\t" + target_text + "\n"
    # sube las líneas a  las listas
    input_texts.append(input_text)
    target_texts.append(target_text)
  
    # completa los conjuntos de caracteres si es necesario
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    i=i+1

# Convierte los dos conjuntos de caracteres
# en dos listas ordenadas
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))  
# calcule el número de tokens (caracteres) en ambos lados
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# calcula la máxima longitud de las secuencias en cada lado
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("preparando datos...")
# crea diccionarios de tokens
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# crea los tensores 1-hot para el encoder y el decoder
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")


for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print ("....\ndatos preparados")    

I: Go.- T: Ve.
I: Go.- T: Vete.
I: Go.- T: Vaya.
I: Go.- T: Váyase.
I: Hi.- T: Hola.
I: Run!- T: ¡Corre!
I: Run!- T: ¡Corran!
I: Run!- T: ¡Huye!
I: Run!- T: ¡Corra!
I: Run!- T: ¡Corred!
I: Run.- T: Corra.
I: Run.- T: Corred.
I: Who?- T: ¿Quién?
I: Wow!- T: ¡Órale!
I: Duck!- T: ¡Inclínate!
I: Fire!- T: ¡Fuego!
I: Fire!- T: ¡Incendio!
I: Fire!- T: ¡Disparad!
I: Help!- T: ¡Ayuda!
I: Help!- T: ¡Socorro! ¡Auxilio!
Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 84
Max sequence length for inputs: 16
Max sequence length for outputs: 47
preparando datos...
....
datos preparados


## Construye el modelo


### Encoder 

In [8]:
# define una secuencia de entrada y la procesa
encoder_inputs = Input(shape = (None, num_encoder_tokens))

# capa recurrente del encoder
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Descartamos las salidas (encoder_outputs)
# solamente se conserva las memoria de  corto (state_h) y 
# largo plazo(state_c)
encoder_states = [state_h, state_c]
#

### Decoder 

In [9]:
# Configuramos el decoder, usando 'encoder_states' como estado inicial
decoder_inputs = Input(shape= (None, num_decoder_tokens))

# capa recurrente del decoder
# Configuramos nuestro decodificador para devolver secuencias de salida completas,
# y también para devolver estados internos. No usamos los
# estados retornados en el modelo de entrenamiento, pero los usaremos en inferencia.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### Modelo completo

In [10]:
!pip install pydot
!pip install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.3 MB/s eta 0:00:00


In [11]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

calculate parameters:

lstm Layer:
        The LSTM layer has three gates: input gate, forget gate, and output gate.
        The number of parameters in each gate depends on the input size, hidden state size, and bias terms.
        The formula to calculate the number of parameters in each gate is (input_size + hidden_size) * hidden_size + hidden_size.
        In this case, you've mentioned an output shape of (None, 256) for the LSTM layer, which indicates a hidden state size of 256.
        The input size is not explicitly mentioned, but you've mentioned that it's connected to 'input_1[0][0]', which would have an input size of 69.
        So, for each gate in the lstm layer: (69 + 256) * 256 + 256 = 333824 parameters.

lstm_1 Layer:
        This LSTM layer seems to be a sequence-to-sequence style LSTM since it has an output shape of (None, None, 256).
        The number of parameters in each gate is calculated the same way as for the previous lstm layer.
        The input size for the gates of this lstm_1 layer is 256, as the previous lstm layer's output size is connected to it.
        For each gate in the lstm_1 layer: (256 + 256) * 256 + 256 = 349184 parameters.

dense Layer:
        The dense layer is a standard fully connected layer.
        The number of parameters is calculated by (input_size * output_size) + output_size.
        In this case, the input size is 256 (output size of the lstm_1 layer), and the output size is 84.
        So, for the dense layer: (256 * 84) + 84 = 21588 parameters.

Overall, the total number of parameters in your model can be calculated by summing up the parameters from the lstm layer, the lstm_1 layer, and the dense layer: 333824 + 349184 + 21588 = 704596 parameters.

In [12]:
model.summary()
plot_model(model, to_file='../Imagenes/s2s.png', 
           show_shapes=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 69)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 84)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    333,824 │ input_layer[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    349,184 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 84)  │     21,588 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 704,596 (2.69 MB)

 Trainable params: 704,596 (2.69 MB)

 Non-trainable params: 0 (0.00 B)

You must install pydot (`pip install pydot`) for `plot_model` to work.


## Entrena el modelo

In [9]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
125/125 [==============================] - 78s 549ms/step - loss: 1.3912 - accuracy: 0.6931 - val_loss: 1.2173 - val_accuracy: 0.6635
Epoch 2/100
125/125 [==============================] - 61s 486ms/step - loss: 0.9613 - accuracy: 0.7358 - val_loss: 1.0431 - val_accuracy: 0.7125
Epoch 3/100
125/125 [==============================] - 64s 514ms/step - loss: 0.8237 - accuracy: 0.7705 - val_loss: 0.9138 - val_accuracy: 0.7375
Epoch 4/100
125/125 [==============================] - 68s 542ms/step - loss: 0.7340 - accuracy: 0.7869 - val_loss: 0.8399 - val_accuracy: 0.7525
Epoch 5/100
125/125 [==============================] - 68s 543ms/step - loss: 0.6790 - accuracy: 0.7995 - val_loss: 0.7907 - val_accuracy: 0.7652
Epoch 6/100
125/125 [==============================] - 66s 531ms/step - loss: 0.6397 - accuracy: 0.8093 - val_loss: 0.7524 - val_accuracy: 0.7755
Epoch 7/100
125/125 [==============================] - 69s 554ms/step - loss: 0.7032 - accuracy: 0.8005 - val_loss: 0.7959 -

In [10]:
# Save model
model.save("../Saved_Models/s2s")

INFO:tensorflow:Assets written to: ../Saved_Models/s2s\assets


INFO:tensorflow:Assets written to: ../Saved_Models/s2s\assets


## Modelo de inferencia

1. Codifica la entrada y recuperar el estado inicial del decodificador.
2. Ejecuta un paso del decodificador con este estado inicial y un token de "inicio de secuencia" como objetivo. La salida será el próximo token de destino.
3. Repite con el token de destino actual y los estados actuales

In [11]:
#
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup taken index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

## Prueba del modelo

Ahora podemos generar oraciones decodificadas como tales:

In [12]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Oración de entrada:", input_texts[seq_index])
    print("Oración decodificada:", decoded_sentence)

1/1 [==============================] - 0s 181ms/step
-
Oración de entrada: Go.
Oración decodificada: Vete.

1/1 [==============================] - 0s 173ms/step
-
Oración de entrada: Go.
Oración decodificada: Vete.

1/1 [==============================] - 0s 305ms/step
-
Oración de entrada: Go.
Oración decodificada: Vete.

1/1 [==============================] - 0s 204ms/step
-
Oración de entrada: Go.
Oración decodificada: Vete.

1/1 [==============================] - 0s 206ms/step
-
Oración de entrada: Hi.
Oración decodificada: Hola.

1/1 [==============================] - 0s 184ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corre!

1/1 [==============================] - 0s 235ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corre!

1/1 [==============================] - 0s 217ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corre!

1/1 [==============================] - 0s 157ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corre!

1/1 [===========

Incluir 1 capa adicional de LSTM al encoder y decoder.

In [ ]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

latent_dim = 256  # Adjust this according to your needs
num_encoder_tokens = ...  # Define the number of encoder tokens
num_decoder_tokens = ...  # Define the number of decoder tokens

# Define input layers
encoder_inputs = Input(shape=(None, num_encoder_tokens))
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Encoder LSTM layers
encoder_lstm_1 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs_1, state_h_1, state_c_1 = encoder_lstm_1(encoder_inputs)

encoder_lstm_2 = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm_2(encoder_outputs_1)

# Decoder LSTM layers
decoder_lstm_1 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs_1, _, _ = decoder_lstm_1(decoder_inputs, initial_state=[state_h, state_c])

decoder_lstm_2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm_2(decoder_outputs_1)

# Dense layer for output
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
output = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)

# Compile the model and specify the optimizer, loss, etc.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])